## Ejemplo 1: Casting

### 1. Objetivos:
    - Aprender a usar `astype`
    - Aprender a lidiar con errores usando `to_numeric`
    - Aprender a convertir `strings` e `ints` a `datetime`
 
---
    
### 2. Desarrollo:

In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/new_york_times_bestsellers-dirty.csv', index_col=0)

df.head()

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24 00:00:00,1212883200000,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24 00:00:00,1212883200000,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24 00:00:00,1212883200000,4,0,1,22.95
3,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,Descr: An aging porn queens aims to cap her ca...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,2008-05-24 00:00:00,1212883200000,5,0,1,24.95
5,http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,Descr: The Minneapolis detective Lucas Davenpo...,Putnam,PHANTOM PREY,5b4aa4ead3089013507db191,2008-05-24 00:00:00,1212883200000,7,4,3,26.95


Tenemos aquí un dataset donde no todos los tipos de datos han sido deducidos correctamente:

In [4]:
df.dtypes

amazon_product_url              object
author                          object
description                     object
publisher                       object
title                           object
oid                             object
bestsellers_date.numberLong     object
published_date.numberLong        int64
rank.numberInt                  object
rank_last_week.numberInt         int64
weeks_on_list.numberInt          int64
price.numberDouble             float64
dtype: object

Específicamente, tenemos dos columnas con fechas (`bestsellers_date.numberLong` y `published_date.numberLong`)  que tienen tipos `object` e `int64`. También tenemos una columna `rank.numberInt` que no tiene el tipo de dato adecuado.

Podemos usar el método `astype` para pasarle a nuestro `DataFrame` un `diccionario` de conversión. Por ejemplo, vamos a convertir nuestras dos columnas de fechas usando un `diccionario` de conversión. El tipo de dato que usamos para manejar fechas es el llamado `datetime`. Este tipo de dato nos permite manipular fechas y horarios muy eficientemente.

In [5]:
diccionario_de_conversion = {
    'bestsellers_date.numberLong': 'datetime64[ns]',
    'published_date.numberLong': 'datetime64[ns]'
}

In [6]:
temp = df.astype(diccionario_de_conversion)

temp.head()

,amazon_product_url,author,description,publisher,title,oid,bestsellers_date.numberLong,published_date.numberLong,rank.numberInt,rank_last_week.numberInt,weeks_on_list.numberInt,price.numberDouble
0,http://www.amazon.com/The-Host-Novel-Stephenie...,Stephenie Meyer,Descr: Aliens have taken control of the minds ...,"Little, Brown",THE HOST,5b4aa4ead3089013507db18c,2008-05-24,1970-01-01 00:20:12.883200,2,1,3,25.99
1,http://www.amazon.com/Love-Youre-With-Emily-Gi...,Emily Giffin,Descr: A woman's happy marriage is shaken when...,St. Martin's,LOVE THE ONE YOU'RE WITH,5b4aa4ead3089013507db18d,2008-05-24,1970-01-01 00:20:12.883200,3,2,2,24.95
2,http://www.amazon.com/The-Front-Garano-Patrici...,Patricia Cornwell,Descr: A Massachusetts state investigator and ...,Putnam,THE FRONT,5b4aa4ead3089013507db18e,2008-05-24,1970-01-01 00:20:12.883200,4,0,1,22.95
3,http://www.amazon.com/Snuff-Chuck-Palahniuk/dp...,Chuck Palahniuk,Descr: An aging porn queens aims to cap her ca...,Doubleday,SNUFF,5b4aa4ead3089013507db18f,2008-05-24,1970-01-01 00:20:12.883200,5,0,1,24.95
5,http://www.amazon.com/Phantom-Prey-John-Sandfo...,John Sandford,Descr: The Minneapolis detective Lucas Davenpo...,Putnam,PHANTOM PREY,5b4aa4ead3089013507db191,2008-05-24,1970-01-01 00:20:12.883200,7,4,3,26.95


In [7]:
temp.dtypes

amazon_product_url                     object
author                                 object
description                            object
publisher                              object
title                                  object
oid                                    object
bestsellers_date.numberLong    datetime64[ns]
published_date.numberLong      datetime64[ns]
rank.numberInt                         object
rank_last_week.numberInt                int64
weeks_on_list.numberInt                 int64
price.numberDouble                    float64
dtype: object

Como puedes ver, nuestras columnas han sido transformadas. Pero parece que hay un problema, puesto que hay muchísima diferencia de años entre la columna `bestsellers_date` y la columna `published_date`. Esto se debe a que `published_date` está en formato 'milisegundos desde La Época (la medianoche UTC del 1 de enero de 1970)' y `pandas` asume por default que estamos lidiando con nanosegundos.

Para evitar este problema vamos a usar el método `pd.to_datetime` para convertir `published_date`:

In [8]:
pd.to_datetime(df['published_date.numberLong'], unit='ms')

0      2008-06-08
1      2008-06-08
2      2008-06-08
3      2008-06-08
5      2008-06-08
          ...    
3027   2013-05-05
3028   2013-05-05
3029   2013-05-05
3030   2013-05-05
3031   2013-05-05
Name: published_date.numberLong, Length: 2266, dtype: datetime64[ns]

`to_datetime` nos permite especificar las unidades para que la conversión se realice con éxito.

Vamos ahora qué pasa si queremos convertir `rank.numberInt` usando `astype`:

In [9]:
df['rank.numberInt'].astype(int)

ValueError: ignored

No podemos hacerlo porque hay unos valores tipo `string` que no pueden ser convertidos a `int`. Para esto usamos el método `to_numeric`, que nos permite indicar que cuando un error sea encontrado, debe de ser sustituido por un `NaN`:

In [10]:
pd.to_numeric(df['rank.numberInt'], errors='coerce')

0        2.0
1        3.0
2        4.0
3        5.0
5        7.0
        ... 
3027     8.0
3028     9.0
3029    11.0
3030    13.0
3031    14.0
Name: rank.numberInt, Length: 2266, dtype: float64

Vamos a reasignar el resultado al `DataFrame` original:

In [11]:
df['rank.numberInt'] = pd.to_numeric(df['rank.numberInt'], errors='coerce')

Ahora, para convertirlo a tipo `int` podemos eliminar los `NaNs` y luego usar `astype`:

In [12]:
df = df.dropna(axis=0).copy()

In [13]:
df['rank.numberInt'] = df['rank.numberInt'].astype(int)

In [14]:
df.dtypes

amazon_product_url              object
author                          object
description                     object
publisher                       object
title                           object
oid                             object
bestsellers_date.numberLong     object
published_date.numberLong        int64
rank.numberInt                   int64
rank_last_week.numberInt         int64
weeks_on_list.numberInt          int64
price.numberDouble             float64
dtype: object

¡Listo!